In [ ]:
# Data generating script for skeleton visualization

In [5]:
# Settings
import os
import json
import numpy
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # 空间三维画图

# Setting for phoenix
w = 210
h = 260
# Setting for CSL
# w = 720
# h = 800
num_joints = 137
view_joints = [0,1,2,3,4,5,6,7,8,9,12,15,16,17,18]
face_view_joints = numpy.arange(25,95)
lefhand_view_joints = numpy.arange(95,116)
righthand_view_joints = numpy.arange(116,137)
view_joints = face_view_joints
# view_joints = numpy.concatenate([view_joints,face_view_joints,lefhand_view_joints,righthand_view_joints],0)
# [
#     [4,3],
#     [6,7]
# ]
line_pairs = numpy.array(\
[
    [17,15],
    [15,0],
    [0,16],
    [16,18],
    [0,1],
    [2,1],
    [1,5],
    [1,8],
    [4,3],
    [3,2],
    [5,6],
    [6,7],
    [9,8],
    [8,12]
])
hand_pairs = numpy.array(\
[
    [0,1],
    [1,2],
    [2,3],
    [3,4],
    [0,5],
    [5,6],
    [6,7],
    [7,8],
    [0,9],
    [9,10],
    [10,11],
    [11,12],
    [0,13],
    [13,14],
    [14,15],
    [15,16],
    [0,17],
    [17,18],
    [18,19],
    [19,20]
])
lefthand_pairs = hand_pairs + 95
righthand_pairs = hand_pairs + 116
line_pairs = numpy.concatenate([line_pairs,lefthand_pairs,righthand_pairs],0)
# Colors
lightred = [0.93,0.60,0.60]
red = [0.83,0.18,0.18]
orange = [1.0,0.5,0.2]
blue = [0.0,0.66,0.88]
pink = [0.92,0.38,0.57]
purple = [0.70,0.53,1.0]
yellow = [1.0,0.93,0.35]

def read_json(file):
    skeletonDict = json.load(open(file,'r'))
    bodySkeleton = numpy.array(skeletonDict['Body']).squeeze()
    faceSkeleton = numpy.array(skeletonDict['Face']).squeeze()
    leftHandSkeleton = numpy.array(skeletonDict['Left hand']).squeeze()
    rightHandSkeleton = numpy.array(skeletonDict['Right hand']).squeeze()
    skeleton = numpy.concatenate([bodySkeleton, faceSkeleton, leftHandSkeleton, rightHandSkeleton],-2)
    if len(skeleton.shape)>2:
        skeleton = skeleton[0]
    return skeleton[:,:2]

def get_pairs(skeleton, line_pairs):
    pairs = []
    for pair in line_pairs:
        i = pair[0]
        j = pair[1]
        if (skeleton[i]==0).all() or (skeleton[j]==0).all():
            print(pair)
        else:
            pairs.append(pair)
    return numpy.array(pairs)

def get_colors():
    # Paint diffenrent color
    colors = []
    for i in range(num_joints):
        if i in [0,15,16,17,18]:
            color = red
        elif i in [1,8,9,12]:
            color = orange
        elif i in [2,3,4]:
            color = blue
        elif i in [5,6,7]:
            color = pink
        elif 25<=i and i<=94:
            color = lightred
        elif 95<=i and i<=115:
            color = purple
        elif 116<=i and i<=136:
            color = yellow
        colors.append(color)
    return numpy.array(colors)


In [4]:
# generate from json file
skeleton_folder = '/Users/liweijie/SLR/scripts/phoenix/01April_2010_Thursday_heute_default-4'
file_list = os.listdir(skeleton_folder)
file_list.sort()
mat = []
point_indices = []
line_indices = []
for i,file in enumerate(file_list[::10]):
    file_path = os.path.join(skeleton_folder,file)
    skeleton = read_json(file_path)
    skeleton = (skeleton-numpy.array([w/2,h/2]))*2/numpy.array([w,h])
    t = 8*10*(i/len(file_list))*numpy.ones([skeleton.shape[0],1])
    colors = get_colors()
    skeleton_t = numpy.concatenate([skeleton,t,colors],1)
    point_indices_t = i*num_joints+numpy.array(view_joints)
    line_indices_t = i*num_joints+get_pairs(skeleton,line_pairs)
    # Append
    mat.append(skeleton_t)
    point_indices.append(point_indices_t)
    line_indices.append(line_indices_t)
mat = numpy.stack(mat,0)
point_indices = numpy.stack(point_indices,0)
line_indices = numpy.stack(line_indices,0)

# Save as byte data
mat.astype(numpy.float32).tofile('/Users/liweijie/projects/visualize-skeleton/skeleton.data')
point_indices.astype(numpy.uint32).tofile('/Users/liweijie/projects/visualize-skeleton/point_indices.data')
line_indices.astype(numpy.uint32).tofile('/Users/liweijie/projects/visualize-skeleton/line_indices.data')

In [20]:
import numpy
# generate from npy
npy_file = 'vae_generate/recons_000100.npy'
# Shape of data is N x L(32) x J x D
data = numpy.load(npy_file)
data = data[0]
mat = []
point_indices = []
line_indices = []
for i,skeleton in enumerate(data):
    skeleton = (skeleton-numpy.array([w/2,h/2]))*2/numpy.array([w,h])
    t = 2*10*(i/len(data))*numpy.ones([skeleton.shape[0],1])
    colors = get_colors()
    skeleton_t = numpy.concatenate([skeleton,t,colors],1)
    point_indices_t = i*num_joints+numpy.array(view_joints)
    line_indices_t = i*num_joints+line_pairs
    # Append
    mat.append(skeleton_t)
    point_indices.append(point_indices_t)
    line_indices.append(line_indices_t)
mat = numpy.stack(mat,0)
point_indices = numpy.stack(point_indices,0)
line_indices = numpy.stack(line_indices,0)

# Save as byte data
mat.astype(numpy.float32).tofile('/Users/liweijie/projects/visualize-skeleton/skeleton.data')
point_indices.astype(numpy.uint32).tofile('/Users/liweijie/projects/visualize-skeleton/point_indices.data')
line_indices.astype(numpy.uint32).tofile('/Users/liweijie/projects/visualize-skeleton/line_indices.data')

In [22]:
# Compare input & recons
import numpy
id = 100
# generate from npy
npy_file = 'vae_generate/input_%06d.npy'%id
# Shape of data is N x L(32) x J x D
data = numpy.load(npy_file)
data = data[0]
mat = []
point_indices = []
line_indices = []
for i,skeleton in enumerate(data):
    skeleton = (skeleton-numpy.array([w/2,h/2]))*2/numpy.array([w,h])
    t = 2*10*(i/len(data))*numpy.ones([skeleton.shape[0],1])
    colors = get_colors()
    skeleton_t = numpy.concatenate([skeleton,t,colors],1)
    point_indices_t = i*num_joints+numpy.array(view_joints)
    line_indices_t = i*num_joints+line_pairs
    # Append
    mat.append(skeleton_t)
    point_indices.append(point_indices_t)
    line_indices.append(line_indices_t)

bias = 32*num_joints-1
# read recons from npy
npy_file = 'vae_generate/recons_%06d.npy'%id
data = numpy.load(npy_file)
data = data[0]
for i,skeleton in enumerate(data):
    skeleton = (skeleton-numpy.array([w/2,h/2]))*2/numpy.array([w,h])
    skeleton = skeleton + numpy.array([5,0])
    t = 2*10*(i/len(data))*numpy.ones([skeleton.shape[0],1])
    colors = get_colors()
    skeleton_t = numpy.concatenate([skeleton,t,colors],1)
    point_indices_t = i*num_joints+numpy.array(view_joints)+bias
    line_indices_t = i*num_joints+line_pairs+bias
    # Append
    mat.append(skeleton_t)
    point_indices.append(point_indices_t)
    line_indices.append(line_indices_t)

mat = numpy.stack(mat,0)
point_indices = numpy.stack(point_indices,0)
line_indices = numpy.stack(line_indices,0)
# Save as byte data
mat.astype(numpy.float32).tofile('/Users/liweijie/projects/visualize-skeleton/skeleton.data')
point_indices.astype(numpy.uint32).tofile('/Users/liweijie/projects/visualize-skeleton/point_indices.data')
line_indices.astype(numpy.uint32).tofile('/Users/liweijie/projects/visualize-skeleton/line_indices.data')

In [6]:
data

array([[[623.942  , 196.61476],
        [632.6453 , 367.0571 ],
        [542.6342 , 380.7331 ],
        ...,
        [488.37903, 800.7846 ],
        [497.12778, 808.24756],
        [501.53348, 811.8475 ]],

       [[400.01447,  98.98565],
        [391.58875, 255.94797],
        [307.07483, 276.081  ],
        ...,
        [233.03192, 645.0049 ],
        [239.81607, 650.0422 ],
        [246.0996 , 652.34534]],

       [[368.18582, 109.69579],
        [370.55008, 268.8963 ],
        [287.80505, 289.41437],
        ...,
        [268.7408 , 621.2943 ],
        [275.78238, 626.6103 ],
        [281.86456, 628.3458 ]],

       ...,

       [[386.5373 ,  89.74047],
        [392.1592 , 250.58833],
        [315.1845 , 271.31195],
        ...,
        [217.61392, 607.00305],
        [225.12231, 612.0029 ],
        [230.7674 , 615.92926]],

       [[403.2172 ,  82.78423],
        [406.4269 , 245.41507],
        [331.06177, 269.43588],
        ...,
        [210.81773, 633.3669 ],
        [217.27559